# 概念分群[二]_ 方法 _實際使用
LastModified : 20160117_2155

＠選定要使用的W2V Model (只能擇一，頗占記憶體)

In [9]:
import gensim
#1__WIKI
#model = gensim.models.Word2Vec.load("W2V_Wiki_Trastd/wiki.zh.sim.seg.trastd.model")

#2__搜狐新闻数据
#model = gensim.models.Word2Vec.load('W2V_SogouCS/SogouCS_Combine.model')

#3__全网新闻数据
model = gensim.models.Word2Vec.load('W2V_SogouCA/SogouCA_Combine.model')

# 將詞語分類

使用方法：

    W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent,PrintResultDic)
    
    Concept   :[字典]。概念字典。
               格式為：Key  : 概念的名稱(僅命名,無實質意義)
               Value: [陣列],內容是可以代表該概念的詞語
               
    Corpus    :[字串]。[檔名]。存取關鍵字的TXT檔。一行一個。
                注意！檔案內第一行要空著！編碼要是UTF-8
    
    SimLevel  :[浮點數]。[0~1]。相似度門檻。
    
    SimPercent:[浮點數]。[0~1]。相似成數(百分比)門檻。
                必須與概念內詞語相似達到一定程度才能算與此概念相似。
                
    PrintResultDic:[字串]。['T'or'F']。決定是否要印出分類完的概念字典
                   (並且顯示與該概念的相似度)

In [2]:
def W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent,PrintResultDic):
    #SimLevel = 0.25#　　相似程度
    #SimPercent = 0.2#　　相似比率

    import operator
    cnt = []#　　計算餵進來的總詞量
    Expt = []#　　存W2V無法辨識的詞語
    Remain = []#　　存沒有通過門檻的詞語
    ClassingResult = {}

    for z in Concept:#　　把概念標頭寫進字典
        ClassingResult[z] = {}#　　每個字典的內容都是一個字典

    for q in open(Corpus,'r'):#　　把萃取關鍵字的結果讀出來        
        q = q.strip().decode('utf-8')#　　去空白,解碼
        cnt.append(q)#　　把讀取內容寫進一陣列(計算樣本的資料量用)
        try:
            TmpDic = {}#　　暫時性的字典(比對詞語與各概念的相似度)
            SameConcept = ''#　　用來存放概念,當q直接是某概念時使用
            TmpWord = 'None'#　　用來比對詞語是不是與概念完全吻合,若是,None會被修改掉;反之則維持None(初值)

            #print '@ ->「',q,'」的概念分類開始: '

            for a in Concept:#　　讀取各概念標頭
                TmpList = []#　　暫時性的陣列(存放'詞語'vs'概念內各詞語'的相似度)     
                for b in Concept[a]:#　　讀取個概念內的詞語

                    #####完全ㄧ樣#####
                    if q.encode('utf-8') == b:#　　如果等於
                        #print '---目標詞「',q,'」就在【',a,'】的概念裡'
                        TmpWord = b#　　把詞語指定給TmpWord
                        break#　　跳出迴圈(已經知道跟某詞語完全一樣,就不用再比其他的詞語了)
                    #####

                    if model.similarity(q,b.decode('utf-8')) > SimLevel:#　　如果相似度達門檻
                        TmpList.append(model.similarity(q,b.decode('utf-8')))#　　就把相似度寫進TmpList裡面

                #####完全一樣#####        
                if TmpWord != 'None':#　　不是None(初值)->被修改過->要完全相等才會出現此情況
                    SameConcept = a#　　把現在比對到的概念( EX:輕鬆 )記起來
                    break#　　跳出迴圈(已經知道是屬於哪個概念,就不用再比對其他概念了)
                #####

                #####印出q與一概念比較的詳細數據
                #print '  -目標詞「',q,'」與【',a,'】的相似陣列( 顯示內容為已高於相似度門檻 :',SimLevel,')'
                #print TmpList
                #print '  -其占【',a,'】概念內容的\n   比重為 :',len(TmpList),'/',len(Dic[a])
                #print '   比率為 :',
                #print float(len(TmpList))/float(len(Concept[a]))
                #print '*'
                #####

                if float(len(TmpList))/float(len(Concept[a])) > SimPercent:#　　如果達到相似成數的門檻
                    #print '  -「',q,'」已過相似_百分比_門檻 :',SimPercent
                    TmpAvg = 0.0#　　存放平均相似度
                    for c in TmpList:#　　把每個相似度讀出來
                        TmpAvg += c#　　加起來
                    TmpDic[a] = float(TmpAvg)/float(len(TmpList))#　　把'總數'除以'陣列長度',得到平均相似度,
                                                                    #再寫進字典裡,當q與多個概念都有相似度食
                                                                    #可以比較與哪個概念的相似度比較高
                    #print '   與【',a,'】的相似度為: ',TmpDic[a],' -> 進入候選名單'
                    #print '# # # # # # #\n'

            #####完全一樣##### 
            if TmpWord != 'None': 
                ClassingResult[SameConcept][(q.encode('utf-8'))] = 1#　　把q寫進總字典的概念裡(Key:q Value:1)
                continue
            #####

            #####檢視暫存字典#####
            #print '\n___檢視',q,'的暫存字典(TmpDic)___'
            #for k in TmpDic:#　　遍歷暫存字典,看q與個概念的相似度
            #    print k,TmpDic[k]
            #print '== == == == == == == == == == =='
            #####

            if len(TmpDic) > 0:#　　如果暫存字典有任何一概念(表示q'至少'與一個概念相似)
                #print '> >小結 :',q,'成功歸類！',
                for d in TmpDic:#　　遍歷TmpDic
                    if d == sorted(TmpDic.iteritems(),key=operator.itemgetter(1),reverse=True)[0][0]:
                        #如果d = 排序後的第一名 (表示d就是與q最像的概念)
                        if TmpDic[d] > SimLevel:#　　如果"q與概念的相似度"大於相似度門檻
                            #print '類別為 :',d,'\n'
                            ClassingResult[d][q.encode('utf-8')] = TmpDic[d]#　　把q寫進總字典的概念裡(Key:q Value:相似度)
                        else:#　　不太可能發生的情況
                            Remain.append('WHAT!?'+q.encode('utf-8'))                    
            else:#　　q與各概念相似度都沒有達標,無法歸類
                Remain.append(q.encode('utf-8'))

        except KeyError:#　　抓到KeyError( W2V不認識q )的例外處理
            #print '「',q,'」EXCEPT! (KeyError)\n'
            Expt.append(q.encode('utf-8'))#　　把q加進例外的陣列理

    ###########檢視總字典#####
    ComfirmDic=['True','T','t','Yes','Y','y']
    if PrintResultDic in ComfirmDic:
        print '\n+ + + 印出字典看看 + + +'
        for ee in ClassingResult:
            print ee,str(ClassingResult[ee]).decode('string_escape')
        print '+ + + + + + + + + + + +'
    ###########

    print '原總量',len(cnt),'\t|','去重複後',len(set(cnt))
    print '--------------------------------------------------'        

    Scnt = 0#　　計算成功被歸類的數量
    for e in ClassingResult:#　　遍歷各概念
        print '【',e,'】','\t數量:',len(ClassingResult[e])
        for f in ClassingResult[e]:#　　遍歷被歸類到此概念中的詞語　
            print f,#　　若加ClassingResult[e][f]則會印出詞語與概念的相似度
            Scnt += 1
        print '\n'

    print '--------------------------------------------------------------------'
    print '相似度門檻',SimLevel,'\t|','相似成數門檻',SimPercent,'　|',\
          '成功分類',Scnt,'　|','成功分類比率',int(float(Scnt)/float(len(set(cnt)))*100),'%'
    print '--------------------------------------------------------------------\n'

    #在此之前Expt 和 Remain都是List , 都有照順序排好 , 印出來可檢查這些詞的原順序
    #此時把List轉成Set , 去除重複值 , 增加可讀性
    Expt = set(Expt)
    Remain = set(Remain)

    #把W2V不認識的印出來
    print '【W2V無法辨識】\t數量',len(Expt)
    for g in Expt:
        print g,
    print '\n'

    #把沒過門檻值的印出來
    print '【低於門檻值】\t數量',len(Remain)
    for h in Remain:
        print h,
    print '\n'

# 概念字典

In [3]:
Dic = {
'藝文':['藝術','美術','戲劇','音樂'],\
    
'信仰':['信仰','祭祀','祭拜','膜拜','信眾','信奉'],\
    
'美食':['美食','爽口','美味','好吃'],\
'新奇':['新奇','新穎','獵奇','獨特','驚喜'],\

'輕鬆':['輕鬆','休閒','遊憩','悠閒','休憩','消遣','愜意','放鬆'],\
'遊玩':['遊玩','好玩','遊樂'],\
'快樂':['快樂','愉快','開心','歡樂','幸福','美好'],\
'有趣':['有趣','趣味'],\

'脫俗':['脫俗','幽雅','清幽','優雅','氣質','心曠神怡'],\
'樂活':['環保','綠化','清新'],\

'安靜':['安靜','寧靜'],\
'乾淨':['乾淨','整潔','潔淨'],\

'懷舊':['懷舊','復古','情調'],\
'古老':['古老','悠久'],\

'髒亂':['骯髒','很髒','髒亂','低劣','灰塵','破舊'],\

'購物':['商圈','百貨','商場'],\
'特產':['特產','紀念品'],\

'運動':['運動','健康'],\

'漂亮':['漂亮','可愛','美麗','優美'],\
'文青':['清新','優雅','氣質'],\

'壯觀':['雄偉','壯觀','壯麗']
}

In [4]:
W2V_ClassificationMethod2(Dic,'NE_SciTfidf-10-20160117.txt',0.25,0.6,'T')


+ + + 印出字典看看 + + +
遊玩 {'度假': 0.3096833531214933, '遊樂園': 0.32768053245268053, '好玩': 1, '遊樂': 1, '露營': 0.29528809776644405, '遊玩': 1, '碰碰車': 0.3619909262408746, '小棧': 0.28054490940486254}
新奇 {'新穎': 1, '絕佳': 0.32800241524353346, '獨特': 1, '寓教於樂': 0.3450672723377153, '驚喜': 1}
懷舊 {'舊時代': 0.284288152784181, '復古': 1, '音樂會': 0.2620802045676628, '搖滾音樂': 0.2824867789631999, '感傷': 0.3637476105403455, '小品': 0.3290444689010642, '慢活': 0.3506380591235201, '時裝': 0.30392221099051764, '美聲': 0.3023288859771273}
乾淨 {'寬敞': 0.3541934325459053, '整潔': 1, '整齊': 0.3715691872839154, '乾淨': 1, '偏貴': 0.2618165220958864, '低廉': 0.2919366587645278, '溫泉水': 0.3350353776815931, '熱水': 0.35110431905430467, '自然光': 0.34683370320016316, '靜肅': 0.27476020361348097, '親民': 0.3122606265694664, '潔淨': 1, '舒服': 0.36818094265146406}
藝文 {'交響樂': 0.2793796688250831, '戲劇': 1, '藝術': 1, '音樂': 1, '戲曲': 0.440171915721983, '攝影': 0.49005040727639315, '音樂廳': 0.3160506988639727, '創意': 0.3755334073713627, '美術': 1}
信仰 {'虔誠': 0.4300783023603412, '祈禱'

In [5]:
W2V_ClassificationMethod2(Dic,'NE_SciTfidf-10-20160117.txt',0.3,0.67,'F')

原總量 955 	| 去重複後 571
--------------------------------------------------
【 遊玩 】 	數量: 3
遊玩 好玩 遊樂 

【 新奇 】 	數量: 3
新穎 獨特 驚喜 

【 懷舊 】 	數量: 1
復古 

【 乾淨 】 	數量: 3
整潔 潔淨 乾淨 

【 藝文 】 	數量: 7
戲劇 藝術 音樂 戲曲 攝影 創意 美術 

【 信仰 】 	數量: 17
虔誠 城隍 靈驗 神祇 祈福 信仰 習俗 道教 參拜 城隍爺 還願 祈禱 寺廟 祭祀 信徒 神明 廟宇 

【 運動 】 	數量: 1
運動 

【 漂亮 】 	數量: 11
漂亮 勇敢 平凡 喜歡 優美 感傷 深愛 可愛 美麗 年輕 奇怪 

【 脫俗 】 	數量: 3
優雅 清幽 幽雅 

【 髒亂 】 	數量: 5
髒亂 低劣 很髒 灰塵 難以想象 

【 樂活 】 	數量: 2
環保 綠化 

【 壯觀 】 	數量: 6
陡峭 建築風格 古色古香 精美 空氣清新 壁畫 

【 古老 】 	數量: 6
古舊 古老 繁華 古代 歷史悠久 著名 

【 美食 】 	數量: 18
鵝肉 臭豆腐 魯肉 新鮮 舒壓 刈包 美味 吳寶春 意麵 美食 潤餅 花生糖 好吃 豬血 應有盡有 品茶 叫賣 爽口 

【 安靜 】 	數量: 3
寧靜 安靜 溫暖 

【 特產 】 	數量: 5
一應俱全 紀念品 手工藝品 日常用品 工藝品 

【 購物 】 	數量: 6
攤商 購物 兒童樂園 地下商場 購物中心 地下街 

【 有趣 】 	數量: 5
有趣 豐富 熱鬧 童話故事 不為人知 

【 文青 】 	數量: 12
熱情奔放 愉悅 色彩鮮豔 充滿活力 有禮 現代感 精緻 莊重 柔和 異國情調 細緻 親切 

【 快樂 】 	數量: 3
愉快 幸福 歡樂 

【 輕鬆 】 	數量: 6
放鬆 悠閒 休閒 消遣 愜意 輕鬆 

--------------------------------------------------------------------
相似度門檻 0.3 	| 相似成數門檻 0.67 　| 成功分類 126 　| 成功分類比率 22 %
----------------------------------------------

# 使用SougoCS Model

In [7]:
W2V_ClassificationMethod2(Dic,'NE_SciTfidf-10-20160117.txt',0.25,0.6,'F')

原總量 955 	| 去重複後 571
--------------------------------------------------
【 遊玩 】 	數量: 33
品茶 聊天 遊樂區 玩水 必去 度假 乘船 浴場 電影院 登山 露營 長廊 徒步旅行 爬山 度假村 觀光果園 戲水 好玩 看電影 遊樂園 碰碰車 泡溫泉 教育館 健行 遊玩 植物園 遊樂 遊樂場 水上公園 悠遊 國家公園 探險 清境 

【 新奇 】 	數量: 4
新穎 慢活 獨特 驚喜 

【 懷舊 】 	數量: 6
復古 好看 湯屋 義大利式 異國情調 時裝 

【 乾淨 】 	數量: 17
寬敞 沒得說 整潔 整齊 乾淨 池水 溫泉水 燙腳 熱水 沐浴露 也很不錯 小桌子 臥室 乾枯 蛤殼 潔淨 不講究 

【 藝文 】 	數量: 27
音樂會 搖滾音樂 音樂 京劇 觀劇 國寶級 國家劇院 學府 美聲 美術 藝術品 展演 美術館 戲劇 藝文活動 戲曲 當代藝術 創意 交響樂 中國武術 新藝術 藝術 藝文 現代藝術 音樂廳 雕塑 文學 

【 信仰 】 	數量: 21
善男信女 虔誠 神殿 蔣中正 古厝 地藏菩薩 指南宮 國父 祈福 蔣公 信仰 習俗 城隍 宗教 佛教 佛像 寺廟 祭祀 神明 香火鼎盛 信徒 

【 運動 】 	數量: 1
運動 

【 漂亮 】 	數量: 8
漂亮 勇敢 平凡 優美 色彩鮮豔 很漂亮 可愛 美麗 

【 脫俗 】 	數量: 15
美景 綠樹 幽雅 優雅 自然環境 秋日 園林 清幽 繁華都市 自然 生活空間 空氣清新 點綴著 建築風格 河景 

【 髒亂 】 	數量: 7
城隍爺 髒亂 低劣 灰塵 年久失修 很髒 著門 

【 樂活 】 	數量: 2
環保 綠化 

【 壯觀 】 	數量: 20
熱帶植物 渭水 淡水河 值得一看 大海 摩天大樓 奇觀 陡峭 雕像 湖面 微縮 山嶺 故宮 風景 城隍廟 夜景 碧湖 碧潭 拱橋 莊嚴 

【 古老 】 	數量: 26
古老 古色古香 古色 廟宇 神祇 古亭 展示館 遺址 發展史 壁畫 陵墓 歷史文物 歷史悠久 雕刻 陳列館 老街 紅毛城 古舊 豐富 神聖 古蹟 古代 道教 民間信仰 古街 宮殿式 

【 美食 】 	數量: 16
鵝肉 臭豆腐 魯肉 新鮮 葡萄 蚵仔煎 刈包 美味 小食 美食 潤餅 嘴饞 好吃 進餐 

In [8]:
W2V_ClassificationMethod2(Dic,'NE_SciTfidf-10-20160117.txt',0.3,0.67,'F')

原總量 955 	| 去重複後 571
--------------------------------------------------
【 遊玩 】 	數量: 4
遊玩 好玩 徒步旅行 遊樂 

【 新奇 】 	數量: 3
新穎 獨特 驚喜 

【 懷舊 】 	數量: 1
復古 

【 乾淨 】 	數量: 4
寬敞 整潔 潔淨 乾淨 

【 藝文 】 	數量: 18
交響樂 展演 美術館 戲劇 音樂會 新藝術 藝術 音樂 現代藝術 戲曲 京劇 美聲 藝文活動 音樂廳 當代藝術 文學 創意 美術 

【 信仰 】 	數量: 10
虔誠 地藏菩薩 信仰 習俗 宗教 佛教 寺廟 祭祀 神明 信徒 

【 運動 】 	數量: 1
運動 

【 漂亮 】 	數量: 8
漂亮 熱情奔放 色彩鮮豔 充滿活力 優美 很漂亮 可愛 美麗 

【 脫俗 】 	數量: 3
優雅 清幽 幽雅 

【 髒亂 】 	數量: 4
髒亂 低劣 灰塵 很髒 

【 樂活 】 	數量: 2
環保 綠化 

【 壯觀 】 	數量: 13
美景 陡峭 威嚴 宮殿式 奇觀 綠樹 熱帶植物 山嶺 風景 夜景 碧湖 碧潭 香火鼎盛 

【 古老 】 	數量: 12
展示館 古老 發展史 古蹟 歷史文物 古色古香 古代 道教 歷史悠久 古亭 民間信仰 古街 

【 美食 】 	數量: 12
鵝肉 臭豆腐 新鮮 蚵仔煎 刈包 美味 小食 美食 潤餅 好吃 花生糖 爽口 

【 安靜 】 	數量: 8
寧靜 清靜 鬧市 安靜 繁華都市 生活空間 空氣清新 熱鬧 

【 特產 】 	數量: 8
手工藝品 翠玉白菜 茶具 雕刻 紀念品 食品街 花卉 工藝品 

【 購物 】 	數量: 2
購物中心 南海路 

【 有趣 】 	數量: 2
有趣 有創意 

【 文青 】 	數量: 9
舒適感 現代感 雅緻 沁涼 精緻 莊重 柔和 異國情調 慢活 

【 快樂 】 	數量: 5
童年 幸福 愉快 溫暖 歡樂 

【 輕鬆 】 	數量: 6
放鬆 悠閒 休閒 消遣 愜意 輕鬆 

--------------------------------------------------------------------
相似度門檻 0.3 	| 相似成數門檻 0.67 　| 成功分類 135 　| 成功分類比率 23 %
-

# 使用SougoCA Model

In [10]:
W2V_ClassificationMethod2(Dic,'NE_SciTfidf-10-20160117.txt',0.25,0.6,'F')

原總量 955 	| 去重複後 571
--------------------------------------------------
【 遊玩 】 	數量: 22
遊樂區 玩水 美景 購物 電影院 動物園 露營 度假村 戲水 好玩 看電影 遊樂園 兒童樂園 碰碰車 泡溫泉 浴場 遊玩 遊樂 植物園 遊樂場 免費參觀 探險 

【 新奇 】 	數量: 4
新穎 獨特 寓教於樂 驚喜 

【 懷舊 】 	數量: 16
舊時代 菊展 新藝術 色彩鮮豔 舞會 現代感 感傷 雅緻 復古 精緻 容易親近 夏初 觀劇 莊重 異國情調 點綴著 

【 乾淨 】 	數量: 9
整潔 整齊 乾淨 舒壓 熱水 沐浴露 空氣清新 乾枯 潔淨 

【 藝文 】 	數量: 20
交響樂 文學 展演 國家劇院 戲劇 音樂會 搖滾音樂 藝術 音樂 現代藝術 戲曲 京劇 美聲 音樂廳 小品 雕刻 當代藝術 藝文 創意 美術 

【 信仰 】 	數量: 23
虔誠 古厝 信仰 宗教 寺廟 紀念堂 香火鼎盛 廟宇 翠玉白菜 神明 善男信女 國父 城隍 保生大帝 神殿 壁畫 祭祀 孫中山 佛教 忠烈祠 佛像 陵墓 信徒 

【 運動 】 	數量: 1
運動 

【 漂亮 】 	數量: 9
漂亮 熱情奔放 優美 很漂亮 童話故事 好看 可愛 美麗 勇敢 

【 脫俗 】 	數量: 11
綠樹 舒適感 優雅 秋日 沁涼 悠遊 清幽 幽雅 生活空間 義大利式 古色 

【 髒亂 】 	數量: 9
古舊 完好如新 髒亂 低劣 著門 灰塵 不講究 很髒 年久失修 

【 樂活 】 	數量: 7
環保 噪聲 自然環境 自然光 熱帶植物 綠化 教育館 

【 壯觀 】 	數量: 20
紀念館 陡峭 長廊 地下城 大海 摩天大樓 奇觀 雕像 湖面 微縮 山嶺 古色古香 莊嚴 古街 夜景 老街 雕塑 故宮 碧潭 綠洲 

【 古老 】 	數量: 17
古代 主題公園 陳列館 遺址 發展史 古蹟 歷史文物 建築風格 習俗 道教 歷史悠久 少數民族 古老 航海 宮殿式 民間信仰 老字號 

【 美食 】 	數量: 18
鵝肉 臭豆腐 魯肉 新鮮 葡萄 嘴饞 蚵仔煎 道地 飢腸轆轆 燙腳 美味 小食 小風 美食 豬血 好吃 進餐 爽口 

【 安靜 】 	數量: 5
繁華都市 寧靜 安靜 清靜 寬敞 

【 特產 】 	

In [11]:
W2V_ClassificationMethod2(Dic,'NE_SciTfidf-10-20160117.txt',0.3,0.67,'F')

原總量 955 	| 去重複後 571
--------------------------------------------------
【 遊玩 】 	數量: 3
遊玩 好玩 遊樂 

【 新奇 】 	數量: 4
新穎 獨特 寓教於樂 驚喜 

【 懷舊 】 	數量: 4
舊時代 異國情調 復古 秋日 

【 乾淨 】 	數量: 3
整潔 潔淨 乾淨 

【 藝文 】 	數量: 13
交響樂 文學 展演 戲劇 藝術 音樂 現代藝術 戲曲 京劇 小品 藝文 創意 美術 

【 信仰 】 	數量: 7
虔誠 信仰 宗教 佛教 祭祀 神明 信徒 

【 運動 】 	數量: 1
運動 

【 漂亮 】 	數量: 5
熱情奔放 漂亮 美麗 優美 可愛 

【 脫俗 】 	數量: 6
優雅 色彩鮮豔 清幽 風景 幽雅 點綴著 

【 髒亂 】 	數量: 4
髒亂 低劣 灰塵 很髒 

【 樂活 】 	數量: 2
環保 綠化 

【 壯觀 】 	數量: 8
美景 綠樹 大海 古色古香 奇觀 湖面 古街 夜景 

【 古老 】 	數量: 12
古老 遺址 古蹟 古代 建築風格 自然環境 歷史文物 道教 歷史悠久 民間信仰 發展史 習俗 

【 美食 】 	數量: 17
鵝肉 臭豆腐 魯肉 新鮮 葡萄 蚵仔煎 嘴饞 看電影 美味 小食 飢腸轆轆 美食 豬血 好吃 進餐 應有盡有 爽口 

【 安靜 】 	數量: 5
清靜 寧靜 安靜 空氣清新 寬敞 

【 特產 】 	數量: 4
紀念品 手工藝品 日常用品 茶具 

【 購物 】 	數量: 4
便利店 購物中心 城隍廟 夜市 

【 有趣 】 	數量: 1
有趣 

【 文青 】 	數量: 5
雅緻 柔和 精緻 充滿活力 現代感 

【 快樂 】 	數量: 8
溫暖 愉悅 享受 令人愉快 愉快 幸福 熱鬧 歡樂 

【 輕鬆 】 	數量: 6
放鬆 悠閒 休閒 消遣 愜意 輕鬆 

--------------------------------------------------------------------
相似度門檻 0.3 	| 相似成數門檻 0.67 　| 成功分類 122 　| 成功分類比率 21 %
------------------------------------------------------